## 1. LSA 벡터

**LSA (Latent Semantic Analysis, 잠재 의미 분석)** 는 텍스트 데이터에서 단어와 문서 간의 숨겨진 의미(잠재 의미)를 파악하는 데 사용되는 통계적 기법입니다. LSA는 특이값 분해(Singular Value Decomposition, SVD)라는 행렬 분해 기법을 사용하여 단어-문서 행렬(term-document matrix)을 차원 축소하고, 이를 통해 단어와 문서의 잠재적인 의미를 나타내는 벡터를 생성합니다. 이 벡터가 바로 **LSA 벡터**입니다.

**LSA 벡터의 특징:**

*   **차원 축소:** 원래의 단어-문서 행렬은 고차원(단어의 수만큼의 차원)이지만, LSA를 통해 저차원(사용자가 지정한 토픽 수만큼의 차원)으로 축소됩니다.
*   **잠재 의미:** LSA 벡터는 단어와 문서의 표면적인 빈도뿐만 아니라, 단어 간의 관계, 문서 간의 관계를 포착하여 잠재적인 의미를 나타냅니다. 예를 들어, "자동차", "엔진", "바퀴"와 같은 단어들은 LSA 벡터 공간에서 서로 가깝게 위치할 가능성이 높습니다.
*   **동의어, 다의어 처리:** LSA는 동의어(의미가 같은 단어)와 다의어(여러 의미를 가진 단어)를 어느 정도 처리할 수 있습니다. 동의어는 비슷한 벡터를 가지게 되고, 다의어는 문맥에 따라 다른 벡터를 가지게 됩니다.
*   **노이즈 감소:** SVD를 통해 차원을 축소하면서 중요하지 않은 정보(노이즈)를 제거하는 효과가 있습니다.




## 2. TruncatedSVD를 이용한 차원 축소

```python
from sklearn.decomposition import TruncatedSVD

def perform_lsa(tfidf_matrix, n_components=3): # n_components : 토픽 수
    """
    TF-IDF 행렬에 대해 LSA를 수행하여 차원을 축소하는 함수.

    Args:
        tfidf_matrix: TF-IDF 행렬 (NumPy 배열 또는 희소 행렬).
        n_components: 축소할 차원(토픽)의 수.

    Returns:
        LSA 벡터 (NumPy 배열), TruncatedSVD 객체.
    """
    svd = TruncatedSVD(n_components=n_components, random_state=42)  # TruncatedSVD 객체 생성, random_state는 재현성을 위한 값.
    lsa_vectors = svd.fit_transform(tfidf_matrix)   # LSA 수행 (차원 축소)
    return lsa_vectors, svd

# 이전 예시에서 생성한 tfidf_matrix를 사용
# lsa_vectors, svd = perform_lsa(tfidf_matrix, n_components=3)  # 예시: 3차원으로 축소

```

**설명:**

*   `TruncatedSVD`: sklearn의 `TruncatedSVD` 클래스를 사용하여 특이값 분해를 수행합니다. TruncatedSVD는 희소 행렬(sparse matrix)에 대해 효율적으로 작동합니다.
*   `n_components`: 축소할 차원의 수 (즉, 토픽의 수)를 지정합니다. 이 값은 하이퍼파라미터이므로 실험을 통해 적절한 값을 찾아야 합니다.
*   `random_state`: 결과의 재현성을 위해 난수 생성기의 시드(seed)를 설정합니다.
*   `svd.fit_transform(tfidf_matrix)`: TF-IDF 행렬에 대해 SVD를 수행하여 차원을 축소하고, LSA 벡터를 반환합니다.



## 3. LSA를 이용한 영화별 토픽 분류 및 결과 확인

```python
# 이전 코드에서 tfidf_matrix와 documents_tokens 가져오기
# (영화 데이터, preprocess_synopsis, create_tfidf_matrix 함수 등)

# 예시 데이터 및 전처리 (위의 코드에서 가져옴)
# 영화 데이터 (시놉시스, 장르, 배우)
movies = [
    {
        "synopsis": """
        "나한테 별로 고마워하지 않아도 돼요" 까칠한 어른 윤서
        "한 번 쯤은 자기를 믿어주는 사람이 있으면 좋잖아요" 꿈 없는 청년 수찬

        시청 정기간행물의 인터뷰어 '윤서'에게 사람의 온기는 한여름의 습하고 불쾌한 더위 같은 것.
        그러던 어느 날, 청년 배달원 '수찬'과 실랑이를 벌이고 만다.
        이후 인터뷰 자리에서 우연찮게 다시 만나게 되는데...

        윤서와 수찬, 두 사람의 불편한 만남은 조금씩 서로를 건드린다.
        """,
        "genre": "드라마",
        "actors": ["임선우", "김명찬", "이장유", "박현숙"]
    },
    {
        "synopsis": """
        "선생님, 저랑 사귀실래요?" 적극적인 어른 민주
        "꺼져" 철벽 많은 급식 윤서

        윤서는 학교에서 학생들에게 인기가 매우 많은 선생님이다.
        어느 날, 윤서는 민주로부터 고백을 받게 된다.
        하지만 윤서는 민주를 거절한다.

        윤서와 민주, 두 사람의 아슬아슬한 만남은 계속된다.
        """,
        "genre": "로맨스",
        "actors": ["김민주", "박서준", "이도현"]
    },
    {
       "synopsis": """
        1919년, 3.1 운동 이후 봉오동 전투에서 승리한 독립군의 이야기를 그린 영화.
        """,
        "genre": "액션",
        "actors": ["유해진", "류준열", "조우진"]
    }
]

# 가중치
genre_weight = 3
actor_weight = 3

# 각 영화별 전처리된 토큰 리스트 생성
documents_tokens = [
    preprocess_synopsis(movie["synopsis"], movie["genre"], movie["actors"], genre_weight, actor_weight)
    for movie in movies
]
# TF-IDF 행렬 생성
tfidf_matrix, word_dict = create_tfidf_matrix(documents_tokens)

# LSA 수행 (차원 축소)
lsa_vectors, svd = perform_lsa(tfidf_matrix, n_components=2)  # 2차원으로 축소

# 3-1. fit_transform() 결과 출력 (LSA 벡터)
print("LSA 벡터 (fit_transform 결과):\n", lsa_vectors)

# 3-2. components_ 출력 (토픽-단어 행렬)
print("\n토픽-단어 행렬 (components_):\n", svd.components_)

# 토픽별 주요 단어 확인
n_top_words = 5
feature_names = list(word_dict.keys()) # documents_tokens를 만들 때 사용했던 단어들이 들어있어야 함

for topic_idx, topic in enumerate(svd.components_): # svd.components_ : (토픽 수, 단어 수)
    print(f"\n토픽 #{topic_idx + 1}:")
    # argsort()는 오름차순 정렬, [::-1]로 내림차순으로 변경, [:n_top_words]로 상위 n_top_words개의 인덱스를 가져옴
    top_word_indices = topic.argsort()[::-1][:n_top_words] 
    # 인덱스를 이용하여 단어 가져옴
    top_words = [feature_names[i] for i in top_word_indices]
    print(" ".join(top_words))

```

**설명:**

*   **LSA 수행:**  `perform_lsa` 함수를 사용하여 TF-IDF 행렬을 LSA 벡터로 변환합니다.
*   **`fit_transform()` 결과:** `lsa_vectors`는 각 영화를 저차원(여기서는 2차원)의 토픽 공간에 표현한 벡터입니다. 각 행은 영화를 나타내고, 각 열은 토픽을 나타냅니다.
*   **`components_`:** `svd.components_`는 토픽-단어 행렬을 나타냅니다. 각 행은 토픽을 나타내고, 각 열은 단어를 나타냅니다. 각 셀의 값은 해당 토픽에서 해당 단어의 중요도를 나타냅니다.
*   **토픽별 주요 단어:** 각 토픽에 대해 `components_`에서 가장 높은 값을 가지는 단어를 추출하여 해당 토픽의 주요 단어를 파악할 수 있습니다.

**결과 해석:**

*   `fit_transform()` 결과(LSA 벡터)를 보면, 각 영화가 어떤 토픽에 가까운지 알 수 있습니다.  예를 들어, 첫 번째 영화와 두 번째 영화가 첫 번째 토픽에 더 가깝고, 세 번째 영화는 두 번째 토픽에 더 가깝다면, 첫 번째와 두 번째 영화는 유사하고 세 번째 영화는 이들과 다르다고 해석할 수 있습니다.
*   `components_`와 토픽별 주요 단어를 보면, 각 토픽이 어떤 의미를 가지는지 추론할 수 있습니다. 예를 들어, 첫 번째 토픽의 주요 단어가 "사랑", "연애", "감정" 등이고, 두 번째 토픽의 주요 단어가 "전쟁", "독립", "투쟁" 등이라면, 첫 번째 토픽은 로맨스, 두 번째 토픽은 역사/전쟁 영화와 관련이 있다고 해석할 수 있습니다.


## 4. LSA 벡터로 영화 유사도 구하기

```python
from sklearn.metrics.pairwise import cosine_similarity

# LSA 벡터 간의 코사인 유사도 계산
similarity_matrix = cosine_similarity(lsa_vectors)
print("\nLSA 벡터 기반 유사도 행렬:\n", similarity_matrix)

# 특정 영화와 다른 영화들 간의 유사도 확인 (예: 0번 영화)
movie_index = 0
similarities = similarity_matrix[movie_index]
print(f"\n{movie_index}번 영화와의 유사도:\n", similarities)

# 유사도가 높은 순으로 정렬
recommended_indices = np.argsort(similarities)[::-1]
print(f"\n{movie_index}번 영화와 유사한 영화 (인덱스):\n", recommended_indices)

# 추천 목록 출력 (영화 제목)
# recommended_movies = [movies[i]["title"] for i in recommended_indices if i != movie_index] # title이 없으므로 genre로 대체
recommended_movies = [movies[i]["genre"] for i in recommended_indices if i != movie_index]
print(f"\n{movie_index}번 영화 추천 목록:\n", recommended_movies)

```

**설명:**

*   `cosine_similarity(lsa_vectors)`: LSA 벡터(`lsa_vectors`)를 사용하여 코사인 유사도 행렬을 계산합니다.
*   `similarity_matrix[movie_index]`: 특정 영화(`movie_index`)와 다른 모든 영화 간의 유사도를 가져옵니다.
*  `np.argsort(similarities)[::-1]`: 코사인 유사도 값을 내림차순 정렬하여 추천 영화를 보여준다.
*   `[movies[i]["genre"] for i in recommended_indices if i != movie_index]`: 추천된 영화의 인덱스를 사용하여 영화 제목을 가져옵니다. 자기 자신은 제외합니다.

**결과 해석:**

*   유사도 행렬을 보면, 각 영화 쌍 간의 코사인 유사도를 확인할 수 있습니다. 값이 1에 가까울수록 유사도가 높습니다.
*   특정 영화와의 유사도를 보면, 해당 영화와 다른 영화들의 유사도 순위를 알 수 있습니다.
*   추천 목록은 유사도가 높은 순으로 정렬된 영화 제목을 보여줍니다.

이 코드는 LSA를 사용하여 영화 간의 잠재적인 의미를 파악하고, 이를 기반으로 유사도를 계산하여 추천하는 예시를 보여줍니다.  실제 추천 시스템에서는 더 많은 데이터와 다양한 기법을 사용합니다.
